In [2]:
import simpy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Simulation parameters
SIM_DURATION = 30 * 12 * 60  # 30 days, 12 hours per day (8 AM to 8 PM)
CAPACITY = 200
WILLINGNESS_TO_VISIT = 0.80  # 80% willingness to visit (predefined threshold)
SHOPPING_PROBABILITY = 0.60  # 60% chance that a parked vehicle will shop

# Arrival rates based on the day of the week and time slots
ARRIVAL_RATES = {
    'Sunday': [(8, 12, 75), (12, 17, 250), (17, 20, 250)],
    'Monday': [(8, 12, 75), (12, 17, 250), (17, 20, 250)],
    'Tuesday': [(8, 12, 75), (12, 17, 250), (17, 20, 250)],
    'Wednesday': [(8, 12, 75), (12, 17, 250), (17, 20, 250)],
    'Thursday': [(8, 12, 75), (12, 17, 250), (17, 20, 250)],
    'Friday': [(8, 12, 100), (12, 17, 350), (17, 20, 400)],
    'Saturday': [(8, 12, 100), (12, 17, 350), (17, 20, 400)]
}

# Initialize data for tracking
arrival_pattern = np.zeros((30, 12))  # 30 days, 12 hours per day (8 AM to 8 PM)
parking_space_utilization = np.zeros(30 * 12)  # Track parking space utilization for each time slot

# Additional tracking parameters
parking_durations = []  # List to store parking durations
total_revenue_parking_before_discount = 0
total_revenue_parking_after_discount = 0
total_revenue_shopping = 0
lost_customers_due_to_price = 0
lost_customers_due_to_full = 0
lost_customers_due_to_WTV = 0
revenue_lost_due_to_WTV = 0
charges_before_discount = []
charges_after_discount = []

# Initialize counters
parked_customers = 0  # To track the number of successfully parked customers
spending_data = []  # List to track shopping spending

def get_arrival_rate(day, hour):
    """Return the arrival rate based on the day and hour."""
    for start, end, rate in ARRIVAL_RATES[day]:
        if start <= hour < end:
            return rate
    return 0

def calculate_charge(parking_duration, discount=0):
    """Calculate the charge based on parking duration with ceiling logic and apply discount."""
    if parking_duration < 30:
        return 15 - discount
    else:
        hours = np.ceil(parking_duration / 60)
        return 25 * hours - discount

def calculate_wtv(spending):
    """Calculate the WTV based on shopping amount."""
    x = -1.27056
    y = -0.00159
    return x * 25 + y * spending + 115

def vehicle(env, parking_lot, day_index, hour_index, inter_arrival_time):
    """Simulates vehicle parking behavior with dynamic timing based on parking lot utilization."""
    global total_revenue_parking_before_discount, total_revenue_parking_after_discount
    global lost_customers_due_to_price, lost_customers_due_to_full, parked_customers
    global total_revenue_shopping, lost_customers_due_to_WTV, revenue_lost_due_to_WTV
    global charges_before_discount, charges_after_discount, spending_data

    with parking_lot.request() as request:
        yield request | env.timeout(0)

        if not request.triggered:  # Parking is full
            lost_customers_due_to_full += 1
            return

        # Calculate parking duration based on current utilization
        utilization = parking_lot.count / CAPACITY  # Calculate parking lot utilization
        if utilization > 0.8:
            parking_duration = 30  # High occupancy, reduce parking duration
        elif utilization < 0.2:
            parking_duration = np.random.uniform(60, 120)  # Low occupancy, extend parking duration
        else:
            parking_duration = np.random.exponential(60)  # Standard duration if in normal range

        parking_durations.append(parking_duration)
        charge_before_discount = calculate_charge(parking_duration)
        charges_before_discount.append(charge_before_discount)
        total_revenue_parking_before_discount += charge_before_discount

        # Increment parked customers count
        parked_customers += 1

        # Determine if the customer shops
        if np.random.random() >= SHOPPING_PROBABILITY:
            spending = np.random.exponential(5000)
            total_revenue_shopping += spending
            spending_data.append(spending)  # Track spending for WTV calculation
            wtv_of_arrived_customer = calculate_wtv(spending)

            if wtv_of_arrived_customer < 70:
                lost_customers_due_to_WTV += 1
                revenue_lost_due_to_WTV += charge_before_discount
                return

            # Apply discount based on spending
            if spending > 5000:
                discount = 25 if parking_duration >= 30 else 15
            else:
                discount = 0
            charge_after_discount = max(charge_before_discount - discount, 0)
        else:
            discount = 0
            charge_after_discount = charge_before_discount

        charges_after_discount.append(charge_after_discount)
        total_revenue_parking_after_discount += charge_after_discount

        yield env.timeout(parking_duration)

def arrival_process(env, parking_lot, day, day_index):
    """Simulates the arrival process for a given day."""
    while env.now < (day_index + 1) * 12 * 60:
        current_hour = (env.now // 60) % 12 + 8
        hour_index = int(current_hour - 8)
        rate = get_arrival_rate(day, current_hour)

        if rate > 0:
            inter_arrival_time = np.random.exponential(60 / rate)
            yield env.timeout(inter_arrival_time)
            env.process(vehicle(env, parking_lot, day_index, hour_index, inter_arrival_time))
        else:
            yield env.timeout(1)

def run_simulation():
    """Runs the simulation for 30 days."""
    env = simpy.Environment()
    parking_lot = simpy.Resource(env, capacity=CAPACITY)

    for day_index in range(30):
        day_name = list(ARRIVAL_RATES.keys())[day_index % 7]
        env.process(arrival_process(env, parking_lot, day_name, day_index))
    
    # Run the environment once for the entire simulation duration (30 days, 12 hours per day)
    env.run(until=30 * 12 * 60)

    total_attempted_arrivals = (parked_customers +
                                lost_customers_due_to_price +
                                lost_customers_due_to_full +
                                lost_customers_due_to_WTV)

    avg_parking_charge_before_discount = np.mean(charges_before_discount) if charges_before_discount else 0
    percentage_lost_due_to_price = (lost_customers_due_to_price / total_attempted_arrivals) * 100 if total_attempted_arrivals else 0
    percentage_lost_due_to_full = (lost_customers_due_to_full / total_attempted_arrivals) * 100 if total_attempted_arrivals else 0
    potential_revenue_lost_due_to_price = lost_customers_due_to_price * avg_parking_charge_before_discount
    avg_shopping_amount = np.mean(spending_data) if spending_data else 0
    total_shopping_revenue_lost_due_to_WTV = lost_customers_due_to_WTV * avg_shopping_amount

    print(f"Total attempted arrivals: {total_attempted_arrivals}")
    print(f"Total parked customers: {parked_customers}")
    print(f"Total lost customers due to price dissatisfaction: {lost_customers_due_to_price}")
    print(f"Percentage of customers lost due to price dissatisfaction: {percentage_lost_due_to_price:.2f}%")
    print(f"Potential parking revenue lost due to price dissatisfaction: ₹{potential_revenue_lost_due_to_price:.2f}")
    print(f"Total lost customers due to parking lot full: {lost_customers_due_to_full}")
    print(f"Percentage of customers lost due to parking lot full: {percentage_lost_due_to_full:.2f}%")
    print(f"Total lost customers due to WTV below threshold: {lost_customers_due_to_WTV}")
    print(f"Total parking revenue before discount: ₹{total_revenue_parking_before_discount:.2f}")
    print(f"Total parking revenue after discount: ₹{total_revenue_parking_after_discount:.2f}")
    print(f"Total shopping revenue: ₹{total_revenue_shopping:.2f}")
    print(f"Total shopping revenue lost due to low WTV: ₹{total_shopping_revenue_lost_due_to_WTV:.2f}")

run_simulation()


Total attempted arrivals: 1222511
Total parked customers: 149243
Total lost customers due to price dissatisfaction: 0
Percentage of customers lost due to price dissatisfaction: 0.00%
Potential parking revenue lost due to price dissatisfaction: ₹0.00
Total lost customers due to parking lot full: 1061895
Percentage of customers lost due to parking lot full: 86.86%
Total lost customers due to WTV below threshold: 11373
Total parking revenue before discount: ₹3765735.00
Total parking revenue after discount: ₹3212310.00
Total shopping revenue: ₹299352327.31
Total shopping revenue lost due to low WTV: ₹56993002.85
